In [4]:
# Load Packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import os
import time
import gc

In [5]:
dataset = pd.read_csv(os.getcwd()+'\sales_dir\sales_data.csv')

In [7]:
dataset.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Size,Type_A,Type_B,Type_C,Markdown_total,month
0,1,1,2012-01-06,16567.69,0,49.01,3.157,219.714258,7.348,151315,1,0,0,38166.78,1
1,1,1,2012-01-13,16894.40,0,48.53,3.261,219.892526,7.348,151315,1,0,0,17423.86,1
2,1,1,2012-01-20,18365.10,0,54.11,3.268,219.985689,7.348,151315,1,0,0,10881.77,1
3,1,1,2012-01-27,18378.16,0,54.26,3.290,220.078852,7.348,151315,1,0,0,3524.39,1
4,1,1,2012-02-03,23510.49,0,56.55,3.360,220.172015,7.348,151315,1,0,0,76351.07,2


In [8]:
dataset.columns

Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment', 'Size', 'Type_A', 'Type_B',
       'Type_C', 'Markdown_total', 'month'],
      dtype='object')

In [6]:
dataset = dataset.drop(columns = ['Date'])

In [25]:
!pip install xgboost

In [26]:
import xgboost
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.feature_selection import RFE
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold

In [21]:
# Algorithms

def knn():
    knn = KNeighborsRegressor(n_neighbors=10)
    return knn

def extraTreesRegressor():
    clf = ExtraTreesRegressor(n_estimators =100, max_features = 'auto', verbose = 1, n_jobs = 1)
    return clf

def randomForestRegressor():
    clf = RandomForestRegressor(n_estimators = 100, max_features = 'log2', verbose = 1)
    return clf

def svm():
    clf = SVR(kernel = 'rbf', gamma = 'auto')
    return clf

def nn():
    clf = MLPRegressor(hidden_layer_sizes=(10,), activation = 'relu', verbose = 3)
    return clf

def linear_reg():
    reg = LinearRegression()
    return reg

def xg():
    xg = XGBRegressor()
    return xg

def ridge_reg():
    rid = Ridge()
    return rid

def lasso_reg():
    las = Lasso()
    return las

def predict_(m,test_x):
    return pd.Series(m.predict(test_x))

def model_(modelfn_name):
    #return knn()
    return modelfn_name()
    #return randomForestRegressor()
    #return svm()
    #return nn()
    
    
def train_(train_x, train_y,modelfn_name):
    m = model_(modelfn_name)
    if train_x.shape[0] > 10000:
        length = int(train_x.shape[0]/10000)
        start_idx = 0
        for i in range(length):
            tx_dat = train_x[start_idx:start_idx+10000]
            ty_dat = train_y[start_idx:start_idx+10000]
            m.fit(tx_dat, ty_dat)
            start_idx +=10000
            gc.collect()
    else:
        m.fit(train_x,train_y)
    #m.fit(train_x,train_y)
    return m

def train_and_predict(train_x, train_y, test_x,modelfn_name):
    m = train_(train_x,train_y,modelfn_name)
    return predict_(m,test_x), m

def calculate_error(test_y,predicted, weights):
    # Print out the MAE, MSE & RMSE
    print("MAE: ", mean_absolute_error(test_y, predicted,weights)) #MAE
    print("MSE: ", mean_squared_error(test_y, predicted,weights)) #MSE
    print("RMSE: ", np.sqrt(mean_squared_error(test_y, predicted, weights))) #RMSE
    print("R2: ", r2_score(test_y, predicted,weights))# RSquared
    return mean_absolute_error(test_y, predicted, weights)

In [9]:
kf = KFold(n_splits=5)
splits = []

for name,group in dataset.groupby(["Store", "Dept"]): # segregating for every store and their respective departments
    group = group.reset_index(drop = True) # setting index to start from 0 and to drop the old index
    train_x = []
    train_y = []
    test_x = []
    test_y = []
    if group.shape[0] <= 5: # for groups less than 5 entries, the rows are marked with variable fold as 0,1,2,3,4
        f = np.array(range(5))
        np.random.shuffle(f)
        group['fold'] = f[:group.shape[0]]
        continue
    fold = 0
    for train_idx, test_idx in (kf.split(group)):
        group.loc[test_idx,'fold'] = fold # All the test rows have values for this variable fold.
        fold +=1
    splits.append(group)
print(len(splits)) # 16065
splits = pd.concat(splits).reset_index(drop=True)

3075


In [10]:
dataset = None

In [11]:
gc.collect()

120

In [12]:
#models


best_model = None
error_cv = 0
best_error = np.iinfo(np.int32).max
for fold in range(5):
    dataset_train = splits.loc[splits['fold'] != fold]
    dataset_test = splits.loc[splits['fold'] == fold]

    train_y = dataset_train['Weekly_Sales']
    train_x = dataset_train.drop(columns=['Weekly_Sales', 'fold'])
    test_y = dataset_test['Weekly_Sales']
    test_x = dataset_test.drop(columns=['Weekly_Sales', 'fold'])
    print(dataset_train.shape, dataset_test.shape)
    dataset_train = None
    dataset_test = None
    gc.collect()
    predicted, model = train_and_predict(train_x, train_y, test_x,knn)
    weights = test_x['IsHoliday'].replace(True, 5).replace(False, 1)
    error = calculate_error(test_y, predicted, weights)
    error_cv += error
    print(fold, error)
    print('----------------------------')
    if error < best_error:
        print('Find best model')
        best_error = error
        best_model = model
error_cv /= 5

(100454, 15) (26662, 15)
MAE:  12013.703836596897
MSE:  420253491.0947748
RMSE:  20500.085148476206
R2:  0.2964639410857689
0 12013.703836596897
----------------------------
Find best model
(100524, 15) (26592, 15)
MAE:  5528.1504545454545
MSE:  59484503.33383972
RMSE:  7712.619745186438
R2:  -64.8914373555292
1 5528.1504545454545
----------------------------
Find best model
(100605, 15) (26511, 15)
MAE:  24795.431999999997
MSE:  1288123560.1418812
RMSE:  35890.43828294496
R2:  -605842.9399228543
2 24795.431999999997
----------------------------
(103416, 15) (23700, 15)
MAE:  9729.405714285715
MSE:  168066277.49750796
RMSE:  12964.037854677375
R2:  -359.77493601654953
3 9729.405714285715
----------------------------
(103465, 15) (23651, 15)
MAE:  11334.57279295922
MSE:  360670098.31960887
RMSE:  18991.31639248867
R2:  0.34833586717733356
4 11334.57279295922
----------------------------


In [13]:
error_cv, best_error

(12680.252959677455, 5528.1504545454545)

In [12]:
# ExtraTreesRegressor

best_model = None
error_cv = 0
best_error = np.iinfo(np.int32).max
for fold in range(5):
    dataset_train = splits.loc[splits['fold'] != fold]
    dataset_test = splits.loc[splits['fold'] == fold]
    
    train_y = dataset_train['Weekly_Sales']
    train_x = dataset_train.drop(columns=['Weekly_Sales', 'fold'])
    test_y = dataset_test['Weekly_Sales']
    test_x = dataset_test.drop(columns=['Weekly_Sales', 'fold'])
    print(dataset_train.shape, dataset_test.shape)
    dataset_train = None
    dataset_test = None
    gc.collect()
    predicted, model = train_and_predict(train_x, train_y, test_x,extraTreesRegressor)
    weights = test_x['IsHoliday'].replace(True, 5).replace(False, 1)
    error = calculate_error(test_y, predicted, weights)
    error_cv += error
    print(fold, error)
    if error < best_error:
        print('Find best model')
        best_error = error
        best_model = model
error_cv /= 5

print(error_cv, best_error)

(100454, 15) (26662, 15)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

MAE:  8287.809934571234
MSE:  245065863.8656965
RMSE:  15654.579645129297
R2:  0.5897412497648915
0 8287.809934571234
Find best model
(100524, 15) (26592, 15)


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.1s finished
[Parallel(n_jobs=1)]: Us

MAE:  6063.65538181818
MSE:  175855640.91666666
RMSE:  13261.057307645822
R2:  -193.79663269685764
1 6063.65538181818
Find best model
(100605, 15) (26511, 15)


...

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Us

MAE:  4635.91131904762
MSE:  179138026.41713896
RMSE:  13384.245455651915
R2:  -383.5418068460079
3 4635.91131904762
(103465, 15) (23651, 15)


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Us

MAE:  8389.9679830603
MSE:  249771108.88062197
RMSE:  15804.14847059537
R2:  0.5487098214374013
4 8389.9679830603


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


In [13]:
# randomForestRegressor

best_model = None
error_cv = 0
best_error = np.iinfo(np.int32).max
for fold in range(5):
    dataset_train = splits.loc[splits['fold'] != fold]
    dataset_test = splits.loc[splits['fold'] == fold]
    
    train_y = dataset_train['Weekly_Sales']
    train_x = dataset_train.drop(columns=['Weekly_Sales', 'fold'])
    test_y = dataset_test['Weekly_Sales']
    test_x = dataset_test.drop(columns=['Weekly_Sales', 'fold'])
    print(dataset_train.shape, dataset_test.shape)
    dataset_train = None
    dataset_test = None
    gc.collect()
    predicted, model = train_and_predict(train_x, train_y, test_x,randomForestRegressor)
    weights = test_x['IsHoliday'].replace(True, 5).replace(False, 1)
    error = calculate_error(test_y, predicted, weights)
    error_cv += error
    print(fold, error)
    if error < best_error:
        print('Find best model')
        best_error = error
        best_model = model
error_cv /= 5

print(error_cv, best_error)

(100454, 15) (26662, 15)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

MAE:  9985.000570762999
MSE:  327357441.89297765
RMSE:  18093.02191158176
R2:  0.45197893793655286
0 9985.000570762999
Find best model
(100524, 15) (26592, 15)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

MAE:  6790.491063636363
MSE:  179661178.21588486
RMSE:  13403.774774886546
R2:  -198.01205534480832
1 6790.491063636363
Find best model
(100605, 15) (26511, 15)


...

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Us

MAE:  6530.505130952381
MSE:  159064826.48719364
RMSE:  12612.090488384296
R2:  -340.4522142864249
3 6530.505130952381
Find best model
(103465, 15) (23651, 15)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

MAE:  10084.710981596376
MSE:  306977513.970791
RMSE:  17520.77378344892
R2:  0.4453484323489387
4 10084.710981596376


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [14]:
# svm

best_model = None
error_cv = 0
best_error = np.iinfo(np.int32).max
for fold in range(5):
    dataset_train = splits.loc[splits['fold'] != fold]
    dataset_test = splits.loc[splits['fold'] == fold]
    
    train_y = dataset_train['Weekly_Sales']
    train_x = dataset_train.drop(columns=['Weekly_Sales', 'fold'])
    test_y = dataset_test['Weekly_Sales']
    test_x = dataset_test.drop(columns=['Weekly_Sales', 'fold'])
    print(dataset_train.shape, dataset_test.shape)
    dataset_train = None
    dataset_test = None
    gc.collect()
    predicted, model = train_and_predict(train_x, train_y, test_x,svm)
    weights = test_x['IsHoliday'].replace(True, 5).replace(False, 1)
    error = calculate_error(test_y, predicted, weights)
    error_cv += error
    print(fold, error)
    if error < best_error:
        print('Find best model')
        best_error = error
        best_model = model
error_cv /= 5

print(error_cv, best_error)

(100454, 15) (26662, 15)
MAE:  14397.872504284174
MSE:  731656130.8235947
RMSE:  27049.14288519314
R2:  -0.22484757811085543
0 14397.872504284174
Find best model
(100524, 15) (26592, 15)
MAE:  4738.291121282027
MSE:  23354000.49994658
RMSE:  4832.597696885867
R2:  -24.869404209479452
1 4738.291121282027
Find best model
(100605, 15) (26511, 15)
MAE:  4915.362618533159
MSE:  24162915.44284874
RMSE:  4915.578851249234
R2:  -11363.558761976097
2 4915.362618533159
(103416, 15) (23700, 15)
MAE:  5590.527264270527
MSE:  31719747.95017305
RMSE:  5632.028759707558
R2:  -67.09034035608032
3 5590.527264270527
(103465, 15) (23651, 15)
MAE:  14207.367899029792
MSE:  672466032.6398172
RMSE:  25931.950035425743
R2:  -0.2150216944921053
4 14207.367899029792
8769.884281479937 4738.291121282027


In [15]:
# nn

best_model = None
error_cv = 0
best_error = np.iinfo(np.int32).max
for fold in range(5):
    dataset_train = splits.loc[splits['fold'] != fold]
    dataset_test = splits.loc[splits['fold'] == fold]
    
    train_y = dataset_train['Weekly_Sales']
    train_x = dataset_train.drop(columns=['Weekly_Sales', 'fold'])
    test_y = dataset_test['Weekly_Sales']
    test_x = dataset_test.drop(columns=['Weekly_Sales', 'fold'])
    print(dataset_train.shape, dataset_test.shape)
    dataset_train = None
    dataset_test = None
    gc.collect()
    predicted, model = train_and_predict(train_x, train_y, test_x,nn)
    weights = test_x['IsHoliday'].replace(True, 5).replace(False, 1)
    error = calculate_error(test_y, predicted, weights)
    error_cv += error
    print(fold, error)
    if error < best_error:
        print('Find best model')
        best_error = error
        best_model = model
error_cv /= 5

print(error_cv, best_error)

(100454, 15) (26662, 15)
Iteration 1, loss = 432967936.40040475
Iteration 2, loss = 399756846.49596012
Iteration 3, loss = 399800999.78720820
Iteration 4, loss = 399785693.63468081
Iteration 5, loss = 399755616.39577663
Iteration 6, loss = 399572934.70492619
Iteration 7, loss = 399540717.75882530
Iteration 8, loss = 399442664.53570437
Iteration 9, loss = 399600199.48858464
Iteration 10, loss = 399507877.78876990
Iteration 11, loss = 399501176.88189358
Iteration 12, loss = 399555913.82409215
Iteration 13, loss = 399362512.64775842
Iteration 14, loss = 399528597.02480292
Iteration 15, loss = 399340959.13015467
Iteration 16, loss = 399508142.74466193
Iteration 17, loss = 399733615.14555722
Iteration 18, loss = 399400159.98945415
Iteration 19, loss = 399377264.51864505
Iteration 20, loss = 399398433.66280383
Iteration 21, loss = 399784800.82055354
Iteration 22, loss = 399344260.52675045
Iteration 23, loss = 399428162.00420970
Iteration 24, loss = 399288454.64080524
Iteration 25, loss = 398

Iteration 54, loss = 278406812.65623391
Iteration 55, loss = 278269657.59629053
Iteration 56, loss = 278348135.42928171
Iteration 57, loss = 278461779.69649941
Iteration 58, loss = 278212566.67433912
Iteration 59, loss = 278539299.82581639
Iteration 60, loss = 278777719.16880655
Iteration 61, loss = 278200058.56918442
Iteration 62, loss = 278304373.47324646
Iteration 63, loss = 278155040.20961523
Iteration 64, loss = 278114466.63647532
Iteration 65, loss = 278249970.60445100
Iteration 66, loss = 278227834.83263779
Iteration 67, loss = 278289804.29836339
Iteration 68, loss = 278090048.93146062
Iteration 69, loss = 278066543.10894006
Iteration 70, loss = 278163795.64983791
Iteration 71, loss = 278101597.47112918
Iteration 72, loss = 278020598.27975279
Iteration 73, loss = 278013345.06033915
Iteration 74, loss = 278052162.95189589
Iteration 75, loss = 278193592.71906453
Iteration 76, loss = 278188993.29789764
Iteration 77, loss = 277760855.01622790
Iteration 78, loss = 278224865.79557389


Iteration 98, loss = 209735045.36091146
Iteration 99, loss = 209600428.58757728
Iteration 100, loss = 209652160.10262039
Iteration 101, loss = 209748657.41995242
Iteration 102, loss = 209579091.05962354
Iteration 103, loss = 209689315.46457231
Iteration 104, loss = 209522091.24208090
Iteration 105, loss = 209545717.41341615
Iteration 106, loss = 209562205.45665538
Iteration 107, loss = 209733156.61109167
Iteration 108, loss = 209992931.19324768
Iteration 109, loss = 209394953.80634388
Iteration 110, loss = 209537524.20619443
Iteration 111, loss = 209515322.56389165
Iteration 112, loss = 209387076.64875433
Iteration 113, loss = 209360961.90654197
Iteration 114, loss = 210011362.21424922
Iteration 115, loss = 209510143.25750110
Iteration 116, loss = 209338501.42055929
Iteration 117, loss = 209272848.43318185
Iteration 118, loss = 209260341.23316473
Iteration 119, loss = 209524852.41803297
Iteration 120, loss = 209464637.39205402
Iteration 121, loss = 209088049.04330453
Iteration 122, los

C:\Users\Sonia\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 229220250.68067789
Iteration 2, loss = 189936883.67597345
Iteration 3, loss = 189122050.24908462
Iteration 4, loss = 188488498.10345727
Iteration 5, loss = 187900116.49867681
Iteration 6, loss = 187495783.83929539
Iteration 7, loss = 187095833.72703019
Iteration 8, loss = 186790348.17668930
Iteration 9, loss = 186505654.51967147
Iteration 10, loss = 186605784.23051789
Iteration 11, loss = 186301364.09275433
Iteration 12, loss = 186178339.65531233
Iteration 13, loss = 186162814.56101766
Iteration 14, loss = 186159112.88968605
Iteration 15, loss = 186085410.99959174
Iteration 16, loss = 186032280.04003930
Iteration 17, loss = 186000347.97970504
Iteration 18, loss = 185992349.63906458
Iteration 19, loss = 185923040.81947932
Iteration 20, loss = 186179529.82592821
Iteration 21, loss = 185795557.12147698
Iteration 22, loss = 185857299.99230394
Iteration 23, loss = 186009052.18455085
Iteration 24, loss = 185784153.44405866
Iteration 25, loss = 185851308.94435275
Iteration

...

C:\Users\Sonia\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 227851353.02548301
Iteration 2, loss = 145699478.08982742
Iteration 3, loss = 131855909.87937492
Iteration 4, loss = 128758398.70308530
Iteration 5, loss = 128390375.62955803
Iteration 6, loss = 128197187.59400883
Iteration 7, loss = 127890567.75075823
Iteration 8, loss = 127656813.64983155
Iteration 9, loss = 127474520.95875373
Iteration 10, loss = 127304170.46247576
Iteration 11, loss = 127177176.58536926
Iteration 12, loss = 127031005.18972771
Iteration 13, loss = 126885960.99574395
Iteration 14, loss = 126796233.09769529
Iteration 15, loss = 126649156.41238545
Iteration 16, loss = 126582850.77794880
Iteration 17, loss = 126459309.19892569
Iteration 18, loss = 126395402.67099874
Iteration 19, loss = 126301488.00558901
Iteration 20, loss = 126182421.35911138
Iteration 21, loss = 126145571.02730677
Iteration 22, loss = 126055884.12164070
Iteration 23, loss = 126015685.96443731
Iteration 24, loss = 125944255.62239629
Iteration 25, loss = 125858465.48536582
Iteration

C:\Users\Sonia\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 906668646.23364389
Iteration 2, loss = 542174404.39643729
Iteration 3, loss = 354780806.23289651
Iteration 4, loss = 259104554.65477428
Iteration 5, loss = 244517510.50047129
Iteration 6, loss = 243865690.85553026
Iteration 7, loss = 243504200.42820936
Iteration 8, loss = 243423265.73434365
Iteration 9, loss = 243304655.08720663
Iteration 10, loss = 243212864.04943153
Iteration 11, loss = 243126719.46602070
Iteration 12, loss = 243072315.74222416
Iteration 13, loss = 243073129.39115870
Iteration 14, loss = 243018981.10285401
Iteration 15, loss = 242958730.10360923
Iteration 16, loss = 242926805.99223080
Iteration 17, loss = 242974513.55371290
Iteration 18, loss = 242938725.21466166
Iteration 19, loss = 242822894.84057856
Iteration 20, loss = 242816559.25519520
Iteration 21, loss = 242749832.81571230
Iteration 22, loss = 242751720.73746583
Iteration 23, loss = 242709589.70429781
Iteration 24, loss = 242692280.91234091
Iteration 25, loss = 242689198.68552858
Iteration

C:\Users\Sonia\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [16]:
# linear_reg

best_model = None
error_cv = 0
best_error = np.iinfo(np.int32).max
for fold in range(5):
    dataset_train = splits.loc[splits['fold'] != fold]
    dataset_test = splits.loc[splits['fold'] == fold]
    
    train_y = dataset_train['Weekly_Sales']
    train_x = dataset_train.drop(columns=['Weekly_Sales', 'fold'])
    test_y = dataset_test['Weekly_Sales']
    test_x = dataset_test.drop(columns=['Weekly_Sales', 'fold'])
    print(dataset_train.shape, dataset_test.shape)
    dataset_train = None
    dataset_test = None
    gc.collect()
    predicted, model = train_and_predict(train_x, train_y, test_x,linear_reg)
    weights = test_x['IsHoliday'].replace(True, 5).replace(False, 1)
    error = calculate_error(test_y, predicted, weights)
    error_cv += error
    print(fold, error)
    if error < best_error:
        print('Find best model')
        best_error = error
        best_model = model
error_cv /= 5

print(error_cv, best_error)

(100454, 15) (26662, 15)
MAE:  51632.93834587601
MSE:  3699399632.7157407
RMSE:  60822.6901140992
R2:  -5.193074163809001
0 51632.93834587601
Find best model
(100524, 15) (26592, 15)
MAE:  40751.156985069036
MSE:  2526633756.1529217
RMSE:  50265.631958157275
R2:  -2797.771453626747
1 40751.156985069036
Find best model
(100605, 15) (26511, 15)
MAE:  51539.78197522198
MSE:  3753878991.386281
RMSE:  61268.907215538624
R2:  -1765563.1962519453
2 51539.78197522198
(103416, 15) (23700, 15)
MAE:  33403.005146295676
MSE:  1549289137.255143
RMSE:  39361.01036883
R2:  -3324.7396884550344
3 33403.005146295676
Find best model
(103465, 15) (23651, 15)
MAE:  22829.53098061574
MSE:  776817471.2855248
RMSE:  27871.445446648886
R2:  -0.403565435962995
4 22829.53098061574
Find best model
40031.28268661569 22829.53098061574


In [27]:
# xg

best_model = None
error_cv = 0
best_error = np.iinfo(np.int32).max
for fold in range(5):
    dataset_train = splits.loc[splits['fold'] != fold]
    dataset_test = splits.loc[splits['fold'] == fold]
    
    train_y = dataset_train['Weekly_Sales']
    train_x = dataset_train.drop(columns=['Weekly_Sales', 'fold'])
    test_y = dataset_test['Weekly_Sales']
    test_x = dataset_test.drop(columns=['Weekly_Sales', 'fold'])
    print(dataset_train.shape, dataset_test.shape)
    dataset_train = None
    dataset_test = None
    gc.collect()
    predicted, model = train_and_predict(train_x, train_y, test_x,XGBRegressor)
    weights = test_x['IsHoliday'].replace(True, 5).replace(False, 1)
    error = calculate_error(test_y, predicted, weights)
    error_cv += error
    print(fold, error)
    if error < best_error:
        print('Find best model')
        best_error = error
        best_model = model
error_cv /= 5

print(error_cv, best_error)

(100454, 15) (26662, 15)
MAE:  8797.53845409179
MSE:  269896463.5067527
RMSE:  16428.525907906427
R2:  0.5481729521014078
0 8797.53845409179
Find best model
(100524, 15) (26592, 15)
MAE:  5913.114398443049
MSE:  191387484.8215916
RMSE:  13834.286567134266
R2:  -211.0013744752935
1 5913.114398443049
Find best model
(100605, 15) (26511, 15)
MAE:  2045.4429251708984
MSE:  11754864.286524469
RMSE:  3428.5367558952125
R2:  -5527.672491497649
2 2045.4429251708984
Find best model
(103416, 15) (23700, 15)
MAE:  4625.652011397226
MSE:  146556760.81753868
RMSE:  12106.06297759675
R2:  -313.60211289277987
3 4625.652011397226
(103465, 15) (23651, 15)
MAE:  8088.621259579511
MSE:  224726969.18384466
RMSE:  14990.896210161842
R2:  0.5939599479486593
4 8088.621259579511
5894.073809736495 2045.4429251708984


In [28]:
# ridge_reg

best_model = None
error_cv = 0
best_error = np.iinfo(np.int32).max
for fold in range(5):
    dataset_train = splits.loc[splits['fold'] != fold]
    dataset_test = splits.loc[splits['fold'] == fold]
    
    train_y = dataset_train['Weekly_Sales']
    train_x = dataset_train.drop(columns=['Weekly_Sales', 'fold'])
    test_y = dataset_test['Weekly_Sales']
    test_x = dataset_test.drop(columns=['Weekly_Sales', 'fold'])
    print(dataset_train.shape, dataset_test.shape)
    dataset_train = None
    dataset_test = None
    gc.collect()
    predicted, model = train_and_predict(train_x, train_y, test_x,ridge_reg)
    weights = test_x['IsHoliday'].replace(True, 5).replace(False, 1)
    error = calculate_error(test_y, predicted, weights)
    error_cv += error
    print(fold, error)
    if error < best_error:
        print('Find best model')
        best_error = error
        best_model = model
error_cv /= 5

print(error_cv, best_error)

(100454, 15) (26662, 15)
MAE:  51211.77208831215
MSE:  3637412831.5182123
RMSE:  60310.967754780824
R2:  -5.089303580712587
0 51211.77208831215
Find best model
(100524, 15) (26592, 15)
MAE:  41537.00066016356
MSE:  2631711546.2443175
RMSE:  51300.21000195143
R2:  -2914.166921945735
1 41537.00066016356
Find best model
(100605, 15) (26511, 15)
MAE:  49632.409800385605
MSE:  3457178322.7973814
RMSE:  58797.77481161495
R2:  -1626015.7897781094
2 49632.409800385605
(103416, 15) (23700, 15)
MAE:  32888.508531650885
MSE:  1495063246.7927623
RMSE:  38666.0477265619
R2:  -3208.3371450459567
3 32888.508531650885
Find best model
(103465, 15) (23651, 15)
MAE:  22310.804759238134
MSE:  747557699.1957633
RMSE:  27341.501407123993
R2:  -0.35069844173669096
4 22310.804759238134
Find best model
39516.099167950066 22310.804759238134


In [29]:
# lasso reg

best_model = None
error_cv = 0
best_error = np.iinfo(np.int32).max
for fold in range(5):
    dataset_train = splits.loc[splits['fold'] != fold]
    dataset_test = splits.loc[splits['fold'] == fold]
    
    train_y = dataset_train['Weekly_Sales']
    train_x = dataset_train.drop(columns=['Weekly_Sales', 'fold'])
    test_y = dataset_test['Weekly_Sales']
    test_x = dataset_test.drop(columns=['Weekly_Sales', 'fold'])
    print(dataset_train.shape, dataset_test.shape)
    dataset_train = None
    dataset_test = None
    gc.collect()
    predicted, model = train_and_predict(train_x, train_y, test_x,lasso_reg)
    weights = test_x['IsHoliday'].replace(True, 5).replace(False, 1)
    error = calculate_error(test_y, predicted, weights)
    error_cv += error
    print(fold, error)
    if error < best_error:
        print('Find best model')
        best_error = error
        best_model = model
error_cv /= 5

print(error_cv, best_error)

(100454, 15) (26662, 15)


C:\Users\Sonia\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3683705822770.441, tolerance: 878765655.4543722
  positive)
C:\Users\Sonia\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2272391559841.7397, tolerance: 485020687.97893673
  positive)
C:\Users\Sonia\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2518991164790.5396, tolerance: 636538399.6014284
  positive)
C:\Users\Sonia\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Dualit

MAE:  52536.5637570735
MSE:  3834127515.627921
RMSE:  61920.332005149336
R2:  -5.418618807169305
0 52536.5637570735
Find best model
(100524, 15) (26592, 15)


C:\Users\Sonia\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3751722209368.4604, tolerance: 887422380.8674309
  positive)
C:\Users\Sonia\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1209754072886.6116, tolerance: 283262415.34051543
  positive)
C:\Users\Sonia\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2328587757044.203, tolerance: 495592715.35053986
  positive)
C:\Users\Sonia\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duali

MAE:  44653.56731536238
MSE:  3071311977.3729615
RMSE:  55419.41877512756
R2:  -3401.1156673460932
1 44653.56731536238
Find best model
(100605, 15) (26511, 15)


...

C:\Users\Sonia\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3863140141913.737, tolerance: 908729279.3098483
  positive)
C:\Users\Sonia\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1149915959276.2021, tolerance: 270176684.72483957
  positive)
C:\Users\Sonia\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2233639361817.9004, tolerance: 483721035.7359908
  positive)
C:\Users\Sonia\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Dualit

MAE:  35827.0034627384
MSE:  1785098581.6642323
RMSE:  42250.42699978584
R2:  -3830.9336643408196
3 35827.0034627384
Find best model
(103465, 15) (23651, 15)


C:\Users\Sonia\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3852427948725.5244, tolerance: 903173589.5783219
  positive)
C:\Users\Sonia\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1174234287203.3782, tolerance: 275456372.0542913
  positive)
C:\Users\Sonia\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2261138568716.581, tolerance: 491459733.64055884
  positive)
C:\Users\Sonia\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Dualit

MAE:  23631.516993661262
MSE:  821284516.3678696
RMSE:  28658.06197857541
R2:  -0.4839091586829585
4 23631.516993661262
Find best model
40469.69518722514 23631.516993661262


C:\Users\Sonia\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1993222418891.9949, tolerance: 487937710.99179536
  positive)
